**Using IPython Widgets**

In [1]:
import ipywidgets as widgets
slider = widgets.IntSlider()
display(slider)

StatementMeta(, 5a931420-7e00-454a-926e-021ad7a2b719, 3, Finished, Available)

IntSlider(value=0)

Using widget output

In [3]:
print (slider.value)

StatementMeta(, 5a931420-7e00-454a-926e-021ad7a2b719, 10, Finished, Available)

36


Datetime picker widget

In [4]:
dt=widgets.DatePicker(description='Select the date')
display(dt)

StatementMeta(, 5a931420-7e00-454a-926e-021ad7a2b719, 11, Finished, Available)

DatePicker(value=None, description='Select the date', step=1)

In [5]:
dt.value

StatementMeta(, 5a931420-7e00-454a-926e-021ad7a2b719, 14, Finished, Available)

datetime.date(2024, 1, 25)

More widget examples coulld be found here: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

In [6]:
display(widgets.Dropdown(options=['1', '2', '3'],value='2'))

StatementMeta(, 5a931420-7e00-454a-926e-021ad7a2b719, 15, Finished, Available)

Dropdown(index=1, options=('1', '2', '3'), value='2')

In [7]:
display(widgets.RadioButtons(
    options=['pepperoni', 'pineapple', 'anchovies'],
    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Pizza topping:',
    disabled=False
))

StatementMeta(, 5a931420-7e00-454a-926e-021ad7a2b719, 17, Finished, Available)

RadioButtons(description='Pizza topping:', index=1, options=('pepperoni', 'pineapple', 'anchovies'), value='pi…

StatementMeta(, 5a931420-7e00-454a-926e-021ad7a2b719, 18, Finished, Available)

StatementMeta(, 5a931420-7e00-454a-926e-021ad7a2b719, 19, Finished, Available)

**Extracting source folder and filenames**

Get partition count

In [206]:
dfCustomerSlv=dfCustomerSlv.withColumn('CustomerKey',monotonically_increasing_id())
display(dfCustomerSlv)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, cc2fc46f-cac2-4aec-bdc1-f5a0df076c4e)

**Joining tables to get Foreign keys**

In [207]:
dfOrderSlv=dfOrderSlv.join(dfCustomerSlv,"CustomerID")\
.join(dfDate,dfOrderSlv.OrderDate==dfDate.Date)\
.select('SalesOrderID','DateKey','CustomerKey','TotalAmount','InsertedDateTime','SourceFileName')
display(dfOrderSlv)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 28df0a8c-9e4c-4d6e-8c86-f9628a0184e1)

**Destination writes**

Overwriting small table

In [208]:
dfOrderSlv.write.format('delta').mode('overwrite').saveAsTable('SlvSalesOrderHeader')
dfCustomerSlv.write.format('delta').mode('overwrite').saveAsTable('SlvCustomer')

StatementMeta(, , , Waiting, )

Incremental write for large tables

Staep 1: Create temp view

In [22]:
dfOrderSlv.createOrReplaceTempView('VwOrdersSilver')

StatementMeta(, , , Waiting, )

In [210]:
%%sql
Select * from VwOrdersSilver order by SalesOrderID

StatementMeta(, , , Waiting, )

<Spark SQL result set with 29 rows and 6 fields>

Step 2: Using Merge command

In [211]:
spark.sql('MERGE INTO SlvSalesOrderHeader T USING VwOrdersSilver S \
ON T.SalesOrderID=S.SalesOrderID \
WHEN MATCHED THEN UPDATE SET * \
WHEN NOT MATCHED THEN INSERT *')

StatementMeta(, , , Waiting, )

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

**Calculating Aggregations**

In [215]:
dfOrderGld=spark.table('SlvSalesOrderHeader')\
.groupBy('CustomerKey')\
.agg(sum('TotalAmount').alias('AllTotals'),avg('TotalAmount').alias('AvgSales'))
display(dfOrderGld)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, eba860a0-e908-4026-84ae-7d629bbc37c1)

In [217]:
dfOrderGld.write.format('delta').mode('overwrite').saveAsTable('GldSales')

StatementMeta(, , , Waiting, )

**Creating aggregate views**

In [223]:
spark.sql('CREATE OR REPLACE VIEW VwAggSales AS SELECT CustomerKey,SUM(TotalAmount) As AllSales FROM SlvSalesOrderHeader GROUP BY CustomerKey')

StatementMeta(, , , Waiting, )

DataFrame[]

In [224]:
%%sql
select * from VwAggSales

StatementMeta(, , , Waiting, )

<Spark SQL result set with 29 rows and 2 fields>